# Data Parsing  

## This code takes the raw xml files as input and finds the sentence embedded in the file and also the seven element tuples which are the ouptut for the sentence.

In [ ]:
import xml.etree.ElementTree as ET
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_sentence(xml_data):
  root = ET.XML(xml_data)
  children = root.getchildren()
  #print(children)
  raw_xml = []
  for i,child in enumerate(children):
   if child.tag=='P':
     raw_xml.append(child.getchildren())
  cnt = 0;
  ent_type = set()
  event_type = set()
  sentence = []
  for i, raw_sent in enumerate(raw_xml):
      for j,word in enumerate(raw_sent):
          if word.tag=='W':
              text = word.text.strip()
              if(text.find('http')!=-1 or text.find('url')!=-1):
                continue
              sentence.append(word.text.strip())
              cnt+=1
          else:
              #print(word.tag)
              tags = word.getchildren()
              #print(tags)
              for tag in tags:
                  if tag.tag=='W':
                    sentence.append(tag.text.strip())

  sentence = " ".join(sentence)
  #print(sentence)
  return sentence

In [ ]:
def get_pos(sentence):
  doc = nlp(sentence)
  pos_tags = []
  for token in doc:
    pos_tags.append(token.pos_)
  pos_sent = " ".join(pos_tags)
  return pos_sent

In [ ]:
def get_dep(sentence):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(sentence)
  dep = []
  for token in doc:
    dep.append(token.dep_)
  dep_sent = " ".join(dep)
  return dep_sent

# **Input to Output Tuples**

## **FINAL FUNCTION**

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def return_tuples(sentence):
    final_tuples = []
    event_dict = {}

    for i in range(len(sentence)):
        if sentence[i][0]=='#':
            temp = sentence[i]
            temp = temp.split()
            key = temp[1]
            event_dict[key] = 0

    for i in range(len(sentence)):

        if sentence[i-1][0]!='$' and sentence[i][0]=='$':
            
            temp = sentence[i]
            temp = temp.split()
            temp_id =  temp[1]
            #print(temp_id)
            event_dict[temp_id] += 1
            
            start_ind_event=0
            end_ind_event=0
            
            count=0
            event_word = ''
            for j in range(len(sentence)):
                temp0 = sentence[j]
                temp0 = temp0.split()

                if temp0[0]=='#' and temp0[1]==temp_id:
                    count+=1
                    if count<=1:
                        start_ind_event = j
                        event_word = temp0[2]
                    end_ind_event = j

            start_ind_argument=i
            end_ind_argumnent=i

            k=i
            while(k<len(sentence)):
                if sentence[k][0]=='$':
                    end_ind_argumnent=k
                    k=k+1
                else:
                    break    

            x = sentence[i].split()
            argument_word = x[2]

            output_tuple = [str(start_ind_event), str(end_ind_event), event_word, str(start_ind_argument), str(end_ind_argumnent), 'ARG',argument_word]
            final_tuples.append(output_tuple)
            #print(output_tuple)
    
    if len(event_dict)==0:
        output_tuple = [str(0), str(0), 'NA', str(1), str(1), 'NA','NA']
    
    for key in event_dict:
        if event_dict[key]:
            start=0
            end=0
            count=0
            event_word = ''
            for j in range(len(sentence)):
                if sentence[j][0]=='#' and sentence[j][2]==temp_id:
                    count+=1
                    if count<=1:
                        start = j
                        x = sentence[j].split()
                        event_word = x[2]
                    end = j            
            output_tuple = [str(start), str(end), event_word, str(1), str(1), 'NA','NA']             

    return final_tuples

In [ ]:
def convert_tuple_to_string(final_tuples):
    final_string = ''
    for _tup_ in final_tuples:
        for token in _tup_:
            final_string+=token
            final_string+=' '
        final_string += '| '
    final_string = final_string[0:len(final_string)-3]
    return final_string

In [ ]:
roles = []

In [ ]:
def get_string(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    elements = []
    numbers = []
    sentences = []
    tags = []
    sentences.append('UNK')
    sentences.append('UNK')

    for child in root:
        if child.tag=='P':
            element = []
            number = []
            
            if child.tag not in tags:
                tags.append(child.tag)

            for x in child:    
                if x.tag=='W':
                  word = x.text.strip()
                  if (word.find("http")!=-1 or word.find('url')!=-1):
                    continue

                if x.tag not in tags:
                    tags.append(x.tag)


                element.append(x)
                number.append(1)
                if x.tag=='W':
                    sentences.append(x.text.strip())
                for y in x:
                    
                    if y.tag=='LINK':
                        last_id = y.attrib["EVENT_ARG"]

                    if y.tag=='W' and x.tag[len(x.tag)-5:len(x.tag)]=='EVENT':
                        sentences.append('# '+x.attrib["ID"]+" "+x.tag+":"+x.attrib["TYPE"]+" "+y.text.strip())
                    if y.tag=='W' and x.tag[len(x.tag)-3:len(x.tag)]=='ARG':
                        sentences.append('$ '+str(last_id)+" "+x.tag[0:len(x.tag)-4]+" "+y.text.strip())
                        role = x.tag[0:len(x.tag)-4]
                        if role not in roles:
                          roles.append(role)
    #print(sentences)
    #print(return_tuples(sentences))
    return convert_tuple_to_string(return_tuples(sentences))  

In [ ]:
events = []

In [ ]:
def add_events(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    for child in root:
        for x in child:
            if x.tag[len(x.tag)-5:len(x.tag)]=='EVENT':
                string = x.tag+":"+x.attrib["TYPE"]
                if string not in events:
                    events.append(string)
    return None

In [ ]:
#get_string('file185.xml')

#Using the above functions on train,dev and test sets

In [ ]:
import os
# sentence_file = open('test_sent.txt',"a")
# pointer_file = open('test_pointer.txt',"a")
# dep_file = open('test_dep.txt',"a")
# pos_file = open('test_pos.txt',"a")

sentences = []
pos_tags= []
dep_tags = []
pointers = []
error_files = []
correct_files = []
folder_path = '/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/test'
os.chdir(folder_path)
for file in os.listdir():
  #print(file)
  file_path = f"{folder_path}/{file}"
  xml_data = open(file_path, 'r').read()
  try:
    sentence = read_sentence(xml_data)
  except:
    error_files.append(file)
    continue
  pos_sent = get_pos(sentence)
  dep_sent = get_dep(sentence)
  sentence = "[unused1] [unused2] "+ sentence
  #pos_sent = get_pos(sentence)
  pos_sent = "<UNK> <UNK> "+pos_sent
  #dep_sent = get_dep(sentence)
  dep_sent = "<UNK> <UNK> "+dep_sent
  #pointer_sent = get_string(file_path)
  try:
    pointer_sent = get_string(file_path)
  except:
    error_files.append(file)
    continue
  add_events(file_path)
  #print(sentence)
  # sentence_file.write(sentence +'\n')
  # pos_file.write(pos_sent+'\n')
  # dep_file.write(dep_sent+'\n')
  # pointer_file.write(pointer_sent+'\n')
  correct_files.append(file)
  sentences.append(sentence)
  pos_tags.append(pos_sent)
  dep_tags.append(dep_sent)
  pointers.append(pointer_sent)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


In [ ]:
#file = '/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/test/file400.xml'
# file = open(file,'r').read()
# print(read_sentence(file))

#print(get_string(file))

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_sent.txt','w') as file:
  pass
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_sent.txt','a') as file:
  for sent in sentences:
    file.write(sent+'\n')

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_pointer.txt','w') as file:
  pass
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_pointer.txt','a') as file:
  for sent in pointers:
    file.write(sent+'\n')

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_pos.txt','w') as file:
  pass
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_pos.txt','a') as file:
  for sent in pos_tags:
    file.write(sent+'\n')

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_dep.txt','w') as file:
  pass
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_dep.txt','a') as file:
  for sent in dep_tags:
    file.write(sent+'\n')

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/tested_files.txt','w') as file:
  pass
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/tested_files.txt','a') as file:
  for x in correct_files:
    file.write(x)
    file.write('\n')

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_error_files.txt','w') as file:
  pass
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/test/test_error_files.txt','a') as file:
  for x in error_files:
    file.write(x)
    file.write('\n')

In [ ]:
# with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/ent_type.txt','a') as file:
#   file.write("ARG\n")
#   file.write("NA")

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/event_type.txt','r') as file:
  curr_events = file.readlines()
  curr_events = [x.strip() for x in curr_events]
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/event_type.txt','a') as file:
  for event in events:
    if event not in curr_events:
      file.write(event+'\n')

In [ ]:
#curr_events = [x.strip() for x in curr_events]

In [ ]:
#curr_events

In [ ]:
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/role.txt','r') as file:
  curr_roles = file.readlines()
  curr_roles = [x.strip() for x in curr_roles]
with open('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/raw_data/role.txt','a') as file:
  for role in roles:
    if role not in curr_roles:
      file.write(role+'\n')
  # file.write("NA")

In [ ]:
#sentences[177]

In [ ]:
#sentences[550]

In [ ]:
#correct_files[550]

In [ ]:
#get_string('/content/drive/My Drive/NLP_Term_Project/dataset/english_ee/test/file977.xml')

In [ ]:
roles

['CASUALTIES',
 'PLACE',
 'PARTICIPANT',
 'TIME',
 'REASON',
 'AFTER_EFFECTS',
 'INTENSITY',
 'TEMPERATURE',
 'MAGNITUDE',
 'DEPTH',
 'EPICENTRE',
 'NAME',
 'SPEED',
 'TYPE']

In [ ]:
events

['MAN_MADE_EVENT:SHOOT_OUT',
 'NATURAL_EVENT:FOREST_FIRE',
 'NATURAL_EVENT:AVALANCHES',
 'MAN_MADE_EVENT:NORMAL_BOMBING',
 'MAN_MADE_EVENT:SUICIDE_ATTACK',
 'NATURAL_EVENT:HEAT_WAVE',
 'MAN_MADE_EVENT:TRAIN_COLLISION',
 'MAN_MADE_EVENT:AVIATION_HAZARD',
 'MAN_MADE_EVENT:SURGICAL_STRIKES',
 'MAN_MADE_EVENT:TRANSPORT_HAZARDS',
 'MAN_MADE_EVENT:INDUSTRIAL_ACCIDENT',
 'MAN_MADE_EVENT:FIRE',
 'NATURAL_EVENT:FLOODS',
 'NATURAL_EVENT:LAND_SLIDE',
 'MAN_MADE_EVENT:VEHICULAR_COLLISION',
 'NATURAL_EVENT:STORM',
 'NATURAL_EVENT:HURRICANE',
 'NATURAL_EVENT:CYCLONE',
 'NATURAL_EVENT:EARTHQUAKE',
 'NATURAL_EVENT:HAIL_STORMS',
 'NATURAL_EVENT:TORNADO',
 'NATURAL_EVENT:VOLCANO',
 'MAN_MADE_EVENT:TERRORIST_ATTACK',
 'MAN_MADE_EVENT:ARMED_CONFLICTS',
 'MAN_MADE_EVENT:MISCELLANEOUS',
 'MAN_MADE_EVENT:CRIME',
 'MAN_MADE_EVENT:ACCIDENTS',
 'NATURAL_EVENT:DROUGHT',
 'MAN_MADE_EVENT:RIOTS']